# Multivariate Variational Mode Decomposition (MVMD) — Signal Decomposition Study

## 🧠 What is MVMD?

**MVMD** (Multivariate Variational Mode Decomposition) is an extension of the **Variational Mode Decomposition (VMD)** technique, adapted to **multichannel signals**. It decomposes a multivariate time series into a fixed number of **modes**, where each mode is:

- **Band-limited** (localized in frequency)
- Shared across all channels (common mode shapes)
- Computed using a variational optimization framework (based on Wiener filtering and frequency demodulation)

Unlike classical techniques like EMD or wavelets, **MVMD is non-recursive, robust to noise, and designed for multi-channel analysis**, making it especially suitable for biomedical, geophysical, and sensor-array signals.


## 📈 Why is MVMD important?

- It can **isolate frequency-specific patterns** in complex, noisy, multi-channel data.
- It ensures that **each mode across channels is aligned**, i.e., mode 1 from all channels represents the same frequency band.
- It supports applications like:
  - Brain signal analysis (EEG, fMRI)
  - Fault detection in mechanical systems
  - Cross-spectral studies in sensor arrays


In [10]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import torch
from pymultifracs.simul import mrw
from scipy.stats import pearsonr
from scipy.signal import resample

# Go up two levels from notebooks/vmd/ to reach src/mvmd/
mvmd_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'src', 'mvmd'))
sys.path.append(mvmd_path)

# Now import mvmd_optimized
from mvmd_original import mvmd 

In [ ]:
# # Parameters
# num_samples = 32768  # Length of the signal (number of samples)
# num_channels = 4    # Number of channels

# H = 0.8  # Hurst exponent (controls the fractality)
# lam = np.sqrt(0.7)  # Lambda, intermittency parameter
# L = 1024  # Integral scale
# # sigma = 1  # Standard deviation of noise

# # 1. Generate a shared low-frequency component
# t = np.linspace(0, 1, num_samples)
# shared_freq_signal = np.sin(2 * np.pi * 0.01 * t)  # Low frequency sinusoid

# # 2. Generate different frequency components for each channel
# signal = np.zeros((num_samples, num_channels))

# for i in range(num_channels):
#     if i < 3:  # Use MRW to introduce multifractality in some channels
#         signal[:, i] = mrw(shape=(num_samples, 1), H=H, lam=lam, L=L, sigma=sigma).squeeze()
#     else:  # Use a random frequency component for the other channels
#         noise = np.random.randn(num_samples) * 0.2
#         signal[:, i] = shared_freq_signal + noise


## Experiment 1:

To evaluate the performance of **Multivariate Variational Mode Decomposition (MVMD)**, we created a **controlled synthetic 3-channel signal** with known frequency components. The idea is to simulate a realistic multichannel scenario where all channels share a common base frequency, but each also contains unique features.

### Signal Design:

- **Duration**: 1 second  
- **Sampling rate**: 32,768 Hz (high-resolution to simulate realistic biomedical signals)  
- **Channels**: 3 parallel signals with shared and distinct frequency components

### Signal Composition:

- **Channel 1**:  
  - Common **36 Hz** tone  
  - Added **2 Hz** low-frequency component  
- **Channel 2**:  
  - Common **36 Hz** tone  
  - Added **24 Hz** medium-frequency component  
- **Channel 3**:  
  - Common **36 Hz** tone  
  - Added **Gaussian noise** (random fluctuations)

### Why this setup?

This structure allows us to test whether MVMD can:
- Separate the **shared 36 Hz tone** across all channels into a clean mode
- Isolate the **unique frequencies (2 Hz, 24 Hz)** present in individual channels
- Handle **noisy inputs** by producing stable and meaningful mode decompositions


In [12]:


# Parameters
T = 32768  # Number of time points
t = np.linspace(0, 1, T)  # Time vector
shared_frequency = 36  # Hz
num_channels = 3  # Number of channels (Channels 1, 2, 3)
signal = np.zeros((T, num_channels))  # Create a signal array with zeros

# Channel 1: 36 Hz tone + 2 Hz tone
signal[:, 0] = np.sin(2 * np.pi * shared_frequency * t) + np.sin(2 * np.pi * 2 * t)  # Channel 1

# Channel 2: 36 Hz tone + 24 Hz tone
signal[:, 1] = np.sin(2 * np.pi * shared_frequency * t) + np.sin(2 * np.pi * 24 * t)  # Channel 2

# Channel 3: 36 Hz tone + Noise
signal[:, 2] = np.sin(2 * np.pi * shared_frequency * t) + np.random.normal(0, 0.2, T)  # Channel 3



In [17]:
num_channels = signal.shape[1]
T = signal.shape[0]
offset = 5

channel_colors = ['blue', 'red', 'green', 'orange']

fig = make_subplots(
    rows=num_channels, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
)

for i in range(num_channels):
    fig.add_trace(
        go.Scatter(
            x=np.arange(T),
            y=signal[:, i] + i * offset,
            mode='lines',
            name=f"Channel {i+1}",
            line=dict(width=1, color=channel_colors[i])
        ),
        row=i + 1, col=1
    )
    # Hide y-axis tick labels
    fig.update_yaxes(
        showticklabels=False,
        row=i + 1, col=1
    )

fig.update_layout(
    height=100 * num_channels,
    width=1000,
    title="Multichannel Signal (Colored by Group & Vertically Spaced)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),
    
    # Add global y-axis label using annotation
    annotations=[
        dict(
            text="Amplitude ",
            xref="paper", yref="paper",
            x=-0.06, y=0.5,
            showarrow=False,
            textangle=-90,
            font=dict(size=14)
        )
    ]
)

fig.update_xaxes(title_text="Time", row=num_channels, col=1)
fig.show()


In [18]:
alpha = 2000  # Balancing parameter
tau = 0.  # Time-step of dual ascent
K = 3  # Number of modes to recover
DC = 0  # No DC component
init = 1  # Initialize frequencies uniformly
tol = 1e-7  # Convergence tolerance
max_N = 50  # Maximum iterations

u, u_hat, omega = mvmd(signal, alpha, tau, K, DC, init, tol, max_N)

TypeError: flip(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
print(u.shape)

In [ ]:
print(u_hat.shape)

In [ ]:
# Parameters
K = u.shape[0]  # Number of modes (4)
T = u.shape[2]  # Number of time points (32768)
num_channels = u.shape[1]  # Number of channels (8)

# Desired number of time points after resampling
num_resampled_points = 1000  # You can adjust this value

# Resample each mode and each channel using scipy's resample
u_resampled = np.zeros((K, num_channels, num_resampled_points))

for k in range(K):
    for i in range(num_channels):
        u_resampled[k, i, :] = resample(u[k, i, :].cpu().numpy(), num_resampled_points)

# Time vector for resampled data
x_resampled = np.linspace(0, T-1, num_resampled_points)

# Channel colors (assign colors based on your preferred scheme)
channel_colors = ['blue', 'red', 'green', 'orange', 'purple', 'cyan', 'magenta', 'black']  # Adjust if needed

# Create subplots: 1 for each mode (K rows)
fig, axs = plt.subplots(K, 1, figsize=(12, 3 * K), sharex=True)

# Plot each mode (from u₁ to uₖ) for each channel
for k in range(K):
    for i in range(num_channels):
        axs[k].plot(x_resampled, u_resampled[k, i, :], label=f"Channel {i + 1}", color=channel_colors[i], linewidth=0.8)

    axs[k].set_title(f"Mode u{k + 1}")
    axs[k].set_ylabel("Amplitude")
    axs[k].grid(True)
    axs[k].legend(loc="upper right", fontsize=8)

# Set x-axis label for the last row
axs[K - 1].set_xlabel("Time (samples)")

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


In [ ]:

# Parameters
K = u.shape[0]  # Number of modes (4)
T = u.shape[2]  # Number of time points (32768)
num_channels = u.shape[1]  # Number of channels (3, after removing the 4th channel)

# Time vector (x-axis)
x = np.arange(T)

# Create subplots: 5 rows, 2 columns for the original data and modes
fig, axs = plt.subplots(5, 3, figsize=(12, 12), sharex=True)

# Channel colors (assign colors based on your preferred scheme)
channel_colors = ['blue', 'red', 'green','purple']  # Adjust as needed

# Channel indices for column 1 and 2
channel_indices = [0, 1, 2]

# Define the y-axis limits for all subplots (to be the same for all)
y_limit = (-1.5, 1.5)  # You can adjust this to match the amplitude range you want for all modes

# Loop through the channels (channel 0 for column 1, channel 2 for column 2)
for col_idx, channel in enumerate(channel_indices):
    # Plot original data in the first row (black color)
    axs[0, col_idx].plot(x, signal[:, channel], color='black', label='Original Data')
    axs[0, col_idx].set_title(f"Channel {channel + 1}")
    axs[0, col_idx].set_ylabel('Amplitude')
    axs[0, col_idx].legend(loc='upper right')
    
    # Loop through the modes (u1 to u4) and plot in rows 2 to 5
    for k in range(K):
        axs[k + 1, col_idx].plot(x, u[k, channel, :].cpu().numpy(), label=f"Mode u{k + 1}", color=channel_colors[k], linewidth=0.8)
        axs[k + 1, col_idx].set_ylabel('Amplitude')
        axs[k + 1, col_idx].legend(loc='upper right')

        # Set the same y-axis limits for all subplots
        axs[k + 1, col_idx].set_ylim(y_limit)

# Set x-axis label for the last row (shared)
axs[3, 0].set_xlabel("Time (samples)")
axs[3, 1].set_xlabel("Time (samples)")
axs[3, 2].set_xlabel("Time (samples)")

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

# Use your actual MVMD output
# u.shape should be [K, C, T]
# If it's torch.Tensor, move to CPU and convert
u_np = u.cpu().numpy() if hasattr(u, 'cpu') else u  # [K, C, T]

fs = 32768  # Sampling rate
K, C, T = u_np.shape

# Compute Welch PSD for each mode and channel
f_axis = None
psd_matrix = []

for k in range(K):
    psd_channels = []
    for c in range(C):
        f, Pxx = welch(u_np[k, c, :], fs=fs, nperseg=2048)
        psd_channels.append(Pxx)
    psd_matrix.append(psd_channels)
    if f_axis is None:
        f_axis = f

# Convert to NumPy array: [K, C, F]
psd_matrix = np.array(psd_matrix)

# Plot: dB scale
fig, axs = plt.subplots(
    int(np.ceil(K / 2)), 2, figsize=(10, 2.8 * int(np.ceil(K / 2))), sharex=True
)
axs = axs.ravel()

for k in range(K):
    for c in range(C):
        axs[k].plot(f_axis, 10 * np.log10(psd_matrix[k, c] + 1e-12), label=f'Channel {c+1}')

    axs[k].set_title(f'Mode {k+1}')
    axs[k].set_xlim(0, 80)
    axs[k].set_ylim(-80, 30)
    axs[k].set_xlabel("Frequency (Hz)")
    axs[k].set_ylabel("Power (dB)")
    axs[k].grid(True)
    axs[k].legend()

# Clean up unused subplots if K is odd
for i in range(K, len(axs)):
    fig.delaxes(axs[i])

fig.suptitle("MVMD Decomposition – PSD per Mode (dB scale)", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
from scipy.signal import welch

fs = 32768
K = u.shape[0]
C = u.shape[1]

for k in range(K):
    for c in range(C):
        mode_signal = u[k, c, :].cpu().numpy()
        f, Pxx = welch(mode_signal, fs=fs, nperseg=2048)
        peak_freq = f[np.argmax(Pxx)]
        print(f"Mode {k+1}, Channel {c+1}: Welch peak at {peak_freq:.2f} Hz")


In [ ]:

# Assuming omega is already defined as a tensor or numpy array
# If omega is a PyTorch tensor, ensure it's on the CPU before converting to NumPy
omega_real = omega.real.cpu().numpy()  # Convert omega to real part (NumPy array)

# Number of modes and iterations (samples)
K = omega_real.shape[1]
iterations = omega_real.shape[0]

# Create the plot
plt.figure(figsize=(10, 6))

# Plot each mode (from mode 1 to mode K)
for k in range(K):
    plt.plot(np.arange(iterations), omega_real[:, k], label=f"Mode {k + 1}")

# Set titles and labels
plt.title("Center Frequencies (Omega) for Each Mode")
plt.xlabel("Iteration")
plt.ylabel("Frequency (Hz)")
plt.legend(title="Modes", fontsize=10)
plt.grid(True)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Convert to NumPy if not already
u_np = u.cpu().numpy()  # shape: [K, num_channels, T]

K = u_np.shape[0]
num_channels = u_np.shape[1]
T = u_np.shape[2]

# Normalize each mode per channel
u_norm = np.zeros_like(u_np)
for k in range(K):
    for c in range(num_channels):
        mode = u_np[k, c, :]
        u_norm[k, c, :] = (mode - np.mean(mode)) / np.std(mode)

# Initialize correlation matrix
corr_matrix = np.zeros((K, K))

# Compute cross-correlation between modes, averaged across channels
for i in range(K):
    for j in range(K):
        corr_values = []
        for c in range(num_channels):
            corr = np.corrcoef(u_norm[i, c, :], u_norm[j, c, :])[0, 1]
            corr_values.append(corr)
        corr_matrix[i, j] = np.mean(corr_values)

# Plot correlation matrix with values
plt.figure(figsize=(6, 5))
im = plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(im, label='Average Pearson Correlation')
plt.title('Cross-Correlation Matrix Between Modes\n(Averaged Over Channels)')
plt.xticks(np.arange(K), [f'Mode {k+1}' for k in range(K)])
plt.yticks(np.arange(K), [f'Mode {k+1}' for k in range(K)])
plt.xlabel('Mode')
plt.ylabel('Mode')

# Annotate correlation values
for i in range(K):
    for j in range(K):
        plt.text(j, i, f"{corr_matrix[i, j]:.2f}", ha='center', va='center', color='black', fontsize=10)

plt.grid(False)
plt.tight_layout()
plt.show()


In [ ]:
""" 🧮 Step-by-step: Compute correlation between Mode 1 and Mode 2 for 4 channels

For channel 1:
    corr(u1[:, 0], u2[:, 0])

For channel 2:
    corr(u1[:, 1], u2[:, 1])

For channel 3:
    corr(u1[:, 2], u2[:, 2])

For channel 4:
    corr(u1[:, 3], u2[:, 3])

✅ Total: 4 correlation values
(one per channel, comparing the same channel in both modes)

Then:
    Average the 4 values to get a single correlation score between Mode 1 and Mode 2
 """

### 🧠 MVMD Decomposition – Interpretation Summary

#### 🔹 1. Time-Domain Modes
- **Mode 1** (blue): High-frequency, present in all channels → captures the **shared 36 Hz tone**.
- **Mode 2** (red): Low-frequency, strong in Channels 1 & 3 → isolates the **low-frequency component**, maybe 2 Hz + noise.
- **Mode 3** (green): Mid-frequency, clear in Channel 2 → corresponds to the **24 Hz tone** + noise.

#### 🔹 2. Frequency-Domain PSD (dB)
- **Mode 1**: Peak near **36 Hz** across all channels.
- **Mode 2**: Low-energy peak near **2–5 Hz**.
- **Mode 3**: Peak around **24 Hz**, strongest in Channel 2.

#### 🔹 3. Cross-Correlation Matrix
- Modes are **uncorrelated** (off-diagonal ≈ 0).
- Confirms clean spectral and temporal separation by MVMD.

---

✅ **Conclusion**:  
MVMD successfully decomposed the multi-channel signal into **distinct, interpretable frequency bands**.  
Time-domain and frequency-domain results are consistent, and each mode is well-isolated.


In [ ]:
# Signal parameters
N = 32768  # Number of time points
L = 512    # Integral scale

# Define multifractality parameters for each channel
# Channels 1 & 2 = strong multifractality (high lambda)
# Channels 3 & 4 = weak multifractality (low lambda)
H_values = [0.3, 0.7, 0.3, 0.7]              # Hurst exponents
lambda_values = [0.3, 0.35, 0.05, 0.1]       # Intermittency λ

# Generate each channel with specified multifractal parameters
channels = []
for H, lam in zip(H_values, lambda_values):
    x = mrw((N,1), H=H, lam=lam, L=L)
    channels.append(x)

# Stack into a 2D array: shape (T, 4)
signal = np.stack(channels, axis=1)


## Experiment 2:

To evaluate the behavior of MVMD on **multifractal time series**, we created a **4-channel signal** using the **Multifractal Random Walk (MRW)** model. This setup simulates signals with distinct levels of **multifractality**, allowing us to observe whether MVMD can distinguish between them in a multivariate setting.

### Signal Design:

- **Length**: 32,768 time points  
- **Channels**: 4 independent realizations  
- **Model**: MRW (Multifractal Random Walk, Bacry et al., 2001)  
- **Integral scale**: 512  
- **Sampling method**: Circulant Matrix Embedding (CME)

### Multifractality Parameters:

| Channel | Hurst Exponent (H) | Intermittency (λ) | Interpretation                      |
|---------|--------------------|-------------------|--------------------------------------|
| 1       | 0.3                | 0.30              | Strong multifractality               |
| 2       | 0.7                | 0.35              | Strong multifractality               |
| 3       | 0.3                | 0.05              | Weak multifractality (quasi-monofractal) |
| 4       | 0.7                | 0.10              | Weak multifractality (quasi-monofractal) |

### Why this setup?

This configuration lets us test whether MVMD is able to:
- Separate modes based on **underlying multifractal properties** (not just frequency)
- Distinguish between **strongly vs. weakly multifractal** dynamics
- Adapt to signals with **heterogeneous temporal complexity**

Such signals are representative of **physiological data** (e.g., EEG, fMRI, HRV) where multifractality may vary between regions, conditions, or subjects.


In [ ]:
# Signal parameters
N = 32768  # Number of time points
L = 512    # Integral scale

# Define multifractality parameters for each channel
# Channels 1 & 2 = strong multifractality (high lambda)
# Channels 3 & 4 = weak multifractality (low lambda)
# Define multifractality parameters for each channel (Hurst exponents and lambda values)
H_values = [0.3, 0.7, 0.3, 0.7]             # Hurst exponents
lambda_values = [0.3, 0.35, 0.05, 0.1]      # Intermittency λ

# Generate each channel with specified multifractal parameters using MRW
channels = []
for H, lam in zip(H_values, lambda_values):
    x = mrw((N,1), H=H, lam=lam, L=L)  # Generate MRW signal for each channel
    channels.append(torch.tensor(x).squeeze())  # Convert to tensor and remove unnecessary dimension

# Stack all channels into a tensor (shape: [4, 32768])
signal = torch.stack(channels, dim=0)

In [ ]:

# Plot all 4 channels
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.plot(signal[:, i], label=f'Channel {i+1} (H={H_values[i]}, λ={lambda_values[i]})')
plt.title("🌀 Simulated MRW Signals with Varying Multifractality")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#signal_mvmd = np.expand_dims(signal.T, axis=0)  # also wrong here
signal = np.squeeze(signal)      # removes the last dim → shape: (32768, 4)
signal_mvmd = signal.T  
print(signal.shape)  # Should print: (4, 32768)
print(signal_mvmd.shape)  # Should print: (4, 32768)


In [ ]:

alpha = 2000  # Balancing parameter
tau = 0.  # Time-step of dual ascent
K = 3  # Number of modes to recover
DC = 0  # No DC component
init = 1  # Initialize frequencies uniformly
tol = 1e-7  # Convergence tolerance
max_N = 50  # Maximum iterations

u, u_hat, omega = mvmd(signal, alpha, tau, K, DC, init, tol, max_N)